In [1]:
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
import datetime

%matplotlib inline

# dit is nodig om jupyter in widescreen te gebruiken (#UWHD masterrace:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# meetpost_id's per richting
meetpunten_riGent = [197, 198, 199, 3744, 3745, 3746, 3750, 3751, 3752, 3756, 3757, 3758, 499, 500, 501]
meetpunten_riGent_lane1 = [197, 3744, 3750, 3756, 499]
meetpunten_riGent_lane2 = [198, 3745, 3751, 3757, 500]
meetpunten_riGent_lane3 = [199, 3746, 3752, 3758, 501]
meetpunten_riAnt = [200, 201, 202, 3747, 3748, 3749, 3753, 3754, 3755, 3759, 3760, 3761, 496, 497, 498, 3691, 3692, 3693, 3694]
meetpunten_riAnt_lane1 = [200, 3747, 3753, 3759, 496, 3691]
meetpunten_riAnt_lane2 = [201, 3748, 3754, 3760, 497, 3692]
meetpunten_riAnt_lane3 = [202, 3749, 3755, 3761, 498, 3693]

loop_richting_en_rijstroken = {"Richting Gent - rijstrook 1": meetpunten_riGent_lane1,
                               "Richting Gent - rijstrook 2": meetpunten_riGent_lane2,
                               "Richting Gent - rijstrook 3": meetpunten_riGent_lane3,
                               "Richting Antwerpen - rijstrook 1": meetpunten_riAnt_lane1,
                               "Richting Antwerpen - rijstrook 2": meetpunten_riAnt_lane2,
                               "Richting Antwerpen - rijstrook 3": meetpunten_riAnt_lane3,
                               }

loop_richting_en_rijstroken_ANT = {"Richting Antwerpen - rijstrook 1": meetpunten_riAnt_lane1,
                                   "Richting Antwerpen - rijstrook 2": meetpunten_riAnt_lane2,
                                   "Richting Antwerpen - rijstrook 3": meetpunten_riAnt_lane3,
                                   }

# loop_richting_en_rijstroken_ANT = {"Richting Antwerpen - rijstrook 1": meetpunten_riAnt_lane1}
meetpunten_riGENT_1lus_allerijstroken = {"Richting Gent - Eerste meetlus - 3 rijstroken": [3744,3745,3746]}
meetpunten_riANT_1lus_allerijstroken = {"Richting Antwerpen - Eerste meetlus - 3 rijstroken": [3747,3748,3749]}
meetpunten_riAnt_alle = {"Richting Antwerpen - alle lussen - alle rijstroken": meetpunten_riAnt}

In [3]:
def getFilePathsinFolderByType(folder, type):
    # haalt de filepaths op van alle bestanden (met een bepaalde extensie) in een folder, ook subfolders
    # --------------------------------------------------------------------------------------------------
    filePaths = []
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith(type):
                filePaths.append(os.path.join(root, file))
    return filePaths

In [4]:
def maakVoertuigMasks(df):
    # Maakt masks voor licht vervoer en zwaar vervoer
    # -----------------------------------------------
    mask_car = (df['car_type'] == 'Car') | (df['car_type'] == 'Motorcycle') | (df['car_type'] == 'Van')
    mask_truck = (df['car_type'] == 'Small Truck') | (df['car_type'] == 'Large Truck')
    
    return mask_car, mask_truck

In [5]:
def  agg_df(df, mask_vehicle, selectie_meetpunten, column_name, agg_time, agg_type):
    n = 0
    for meetpost in selectie_meetpunten:
        print('bezig met '+ str(meetpost))
        mask_meetpunt = (df['meetpunt_id'] == meetpost)
        
        if n == 0:
            # Aggregeer naar voertuigtype, richting+lane en tijdsduur
            df_agg = df[(mask_meetpunt & mask_vehicle)].resample(agg_time).agg({column_name: agg_type})
            df_agg[(str(meetpost)+'_'+agg_type)]=df_agg[column_name]
            del df_agg[column_name]
        else:
            df_agg_temp = df[(mask_meetpunt & mask_vehicle)].resample(agg_time).agg({column_name: agg_type})
            df_agg[(str(meetpost)+'_'+agg_type)] = df_agg_temp[column_name]
        n += 1
            
#     df_agg['Totaal'] = df_agg.sum(axis=1)    
    return df_agg

In [6]:
# Lees alle parquetfiles in en schrijf data weg naar dataframes
parquetFilesPaths = getFilePathsinFolderByType(r'C:\Case\Brondata\dataset_02\measurements_v2_2', '.parquet')

n = 0
for filePath in parquetFilesPaths:
    if n == 9:
        # Lees een parquetfile in, filter de benodigde kolommen, zet de meettijd als index = datetimeseries
        # -------------------------------------------------------------------------------------------------
        
        str_index = filePath.find('date=')
        datum_str = filePath[str_index+5:str_index+15]
        print("Bezig met "+datum_str)
        
        df = pq.read_table(filePath).to_pandas().filter( \
            ['meetpunt_id', 'meettijd', 'volgtijd', 'snelheid_kmph', 'elek_lengte_dm', 'car_type'])
        df = df.set_index(['meettijd']).sort_index()
        
        # Maak voertuigmasks
        mask_car, mask_truck = maakVoertuigMasks(df)
                
        for key, value in meetpunten_riAnt_alle.items():
            titel = key
            meetpunten_filter_array = value
            print(" "*2, titel)
            dir_output = datum_str + " " + titel
            
            df_agg_min = agg_df(df, mask_car+mask_truck, meetpunten_filter_array, 'snelheid_kmph', '5min', 'mean')
            df_agg_min_car = agg_df(df, mask_car, meetpunten_filter_array, 'snelheid_kmph', '5min', 'mean')
            df_agg_min_truck = agg_df(df, mask_truck, meetpunten_filter_array, 'snelheid_kmph', '5min', 'mean')
        
    n += 1
    
df_agg_min.to_csv('C:\Case\Output\descr_ana'+'snelheid_totaal_5min.csv',index=True, header=True)
df_agg_min_car.to_csv('C:\Case\Output\descr_ana'+'snelheid_car_5min.csv',index=True, header=True)
df_agg_min_truck.to_csv('C:\Case\Output\descr_ana'+'snelheid_truck_5min.csv',index=True, header=True)

 
    

Bezig met 2015-11-23
   Richting Antwerpen - alle lussen - alle rijstroken
bezig met 200
bezig met 201
bezig met 202
bezig met 3747
bezig met 3748
bezig met 3749
bezig met 3753
bezig met 3754
bezig met 3755
bezig met 3759

C:\Users\koen_\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:194: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  op=op_str, alt_op=unsupported[op_str]



bezig met 3760
bezig met 3761
bezig met 496
bezig met 497
bezig met 498
bezig met 3691
bezig met 3692
bezig met 3693
bezig met 3694
bezig met 200
bezig met 201
bezig met 202
bezig met 3747
bezig met 3748
bezig met 3749
bezig met 3753
bezig met 3754
bezig met 3755
bezig met 3759
bezig met 3760
bezig met 3761
bezig met 496
bezig met 497
bezig met 498
bezig met 3691
bezig met 3692
bezig met 3693
bezig met 3694
bezig met 200
bezig met 201
bezig met 202
bezig met 3747
bezig met 3748
bezig met 3749
bezig met 3753
bezig met 3754
bezig met 3755
bezig met 3759
bezig met 3760
bezig met 3761
bezig met 496
bezig met 497
bezig met 498
bezig met 3691
bezig met 3692
bezig met 3693
bezig met 3694


In [7]:
# Moving Average berekening

column_name_array = []

for column in df_agg_min:
    sma_n = 6
    column_name_array.append(column)
    headerSMA = column+'_SMA'
    df_agg_min[headerSMA] = df_agg_min[column].rolling(window=sma_n).mean()
    
    headerSMAverschil = column+'_SMA_%'
    df_agg_min[headerSMAverschil] = 100.0*(df_agg_min[column]-df_agg_min[headerSMA])/df_agg_min[headerSMA]
    
    
df_agg_min['000_meettijd_proper'] = df_agg_min.index.strftime("%H:%M")
df_agg_min = df_agg_min.reindex(sorted(df_agg_min.columns), axis=1)

print(df_agg_min.head(20))

df_agg_min.to_csv('C:\Case\Output\descr_ana'+'_SMA_test_5min.csv',index=True, header=True)


                    000_meettijd_proper   200_mean  200_mean_SMA  \
meettijd                                                           
2015-11-23 00:00:00               00:00  85.500000           NaN   
2015-11-23 00:05:00               00:05  85.724138           NaN   
2015-11-23 00:10:00               00:10  87.000000           NaN   
2015-11-23 00:15:00               00:15  86.965517           NaN   
2015-11-23 00:20:00               00:20  87.617647           NaN   
2015-11-23 00:25:00               00:25  84.965517     86.295470   
2015-11-23 00:30:00               00:30  85.382353     86.275862   
2015-11-23 00:35:00               00:35  83.758621     85.948276   
2015-11-23 00:40:00               00:40  87.703704     86.065560   
2015-11-23 00:45:00               00:45  85.333333     85.793529   
2015-11-23 00:50:00               00:50  87.647059     85.798431   
2015-11-23 00:55:00               00:55  85.700000     85.920845   
2015-11-23 01:00:00               01:00  88.0526

[20 rows x 58 columns]
